In [25]:
import awswrangler as wr
from datetime import datetime, timedelta
import sagemaker
import boto3
import json

now = datetime.now() - timedelta(hours = 4)

# episodes = ["1716761117540","test"] # pageids of episodes to take
# programs = ["1460018486818"] # program pageids of programs to take

In [4]:
# AWS login to use sagemaker endpoints
session = boto3.Session(profile_name='vrt-analytics-engineer-nonsensitive')
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

In [5]:
query = f"""
SELECT
*
FROM (
SELECT 
*
FROM 
derived_prod.vrtmax_catalog_mediaid_history WHERE year = {now.year} and month = {now.month} and day = {now.day} and hour = {now.hour} 
AND offering_publication_planneduntil > CURRENT_TIMESTAMP
) vrtmax_catalog
LEFT JOIN (
SELECT 
mediacontent_media_id as mediacontent_media_id_duplicate,
array_join(array_agg(subtitle ORDER BY starttc ASC), ' ') as subtitle
FROM (
SELECT 
MIN(subtitle) as subtitle,
mediacontent_media_id,
starttc
FROM (
SELECT 
subtitle,
mediacontent_media_id,
starttc
FROM
(
SELECT 
objid, subtitle
FROM (
SELECT objid, subtitle, 
row_number() OVER (partition by subtitle ORDER BY objid DESC) as rn 
FROM (
SELECT objid, a_vrt_subti2  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti3  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subtit as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s3"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti4 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s4"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_speech as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_sp"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_subti1 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_su"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
)
WHERE rn = 1
) a
LEFT JOIN (
SELECT objid, segmentcontentid, starttc
FROM (
SELECT objid, segmentcontentid, starttc,
row_number() OVER (partition by segmentcontentid ORDER BY objid DESC) as rn 
FROM (
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub3"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub4"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_subt"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
GROUP BY objid, segmentcontentid, starttc
) 
WHERE rn = 1
) b
ON a.objid = b.segmentcontentid
LEFT JOIN (
SELECT objid,  a_vrt_progra ,a_vrt_media_
FROM (
SELECT objid,  a_vrt_progra ,a_vrt_media_,
row_number() OVER (partition by objid ORDER BY a_vrt_media_ DESC) as rn 
FROM "operationaldata_nonsensitive_prod"."depot_dmo_program"  WHERE import_year = 2024 and import_month = 6 and import_day = 9
) WHERE rn = 1
) c
ON b.objid = c.objid
INNER JOIN (
SELECT mediacontent_media_id, 	mediacontent_media_origin_id  
FROM (
SELECT mediacontent_media_id, mediacontent_media_origin_id, row_number() OVER (PARTITION BY mediacontent_media_origin_id   ORDER BY 	contactmoment_ingest_timestamp DESC)  as rn FROM "marketing_prod"."dtr_vd_dms_audio_video_metadata_datariver"
)
WHERE rn = 1
) d
ON c.a_vrt_media_ = d.mediacontent_media_origin_id
)
GROUP BY mediacontent_media_id, starttc
)
GROUP BY mediacontent_media_id
) subtitles  
ON vrtmax_catalog.mediacontent_media_id = subtitles.mediacontent_media_id_duplicate
"""

df = wr.athena.read_sql_query(sql=query, database="derived_prod",boto3_session=session)

In [6]:
# drop columns
df = df.drop(['mediacontent_media_id_duplicate'], axis=1)

In [35]:
# cleaning
df["subtitle"] = df["subtitle"].str.replace('\r', '')
df["subtitle"] = df["subtitle"].str.replace('\n', '')
df["subtitle"] = df["subtitle"].str.replace("-","")
df["offering_publication_planneduntil"] = df["offering_publication_planneduntil"].astype(str)

In [36]:
# Fields to use
fields_to_embed = [
    "subtitle","mediacontent_page_description_program","mediacontent_page_description","mediacontent_page_editorialtitle_program"
]

metadata_fields = [
    "mediacontent_pagetitle_program","mediacontent_pagetitle_season","mediacontent_pagetitle","mediacontent_page_description_program","mediacontent_page_description"
    ,"offering_publication_planneduntil","brand_contentbrand","mediacontent_pageurl","mediacontent_imageurl","mediacontent_programimageurl"
]

In [37]:
# take only  rows where at least one of the fields to be embedded is not null
query_condition = []

for field in fields_to_embed:
    query_condition.append("(" + field + ".notnull())")

query_condition = " | ".join(query_condition)
df = df.query(query_condition)

In [38]:
# create csv + metadata combinations (for every row)
for index, row in df.iterrows():
    df[df.index == index][fields_to_embed].to_csv("opensearch/" + row["mediacontent_media_id"] + ".csv",index=False)
    metadata = dict()
    metadata["metadataAttributes"] = df[df.index == index][metadata_fields].to_dict("records")[0]

    # Convert and write JSON object to file
    with open("opensearch/" + row["mediacontent_media_id"] + ".metadata.json", "w") as outfile: 
        json.dump(metadata, outfile)

    if index > 1:
        break

In [39]:
metadata

{'metadataAttributes': {'mediacontent_pagetitle_program': 'Radio2 aan zee',
  'mediacontent_pagetitle_season': 'Seizoen 2024',
  'mediacontent_pagetitle': 'Radio 2 aan zee - optredens',
  'mediacontent_page_description_program': None,
  'mediacontent_page_description': 'Lekker blijven hangen in Blankenberge met Margriet Hermans!',
  'offering_publication_planneduntil': '2030-01-01 22:59:00.000',
  'brand_contentbrand': None,
  'mediacontent_pageurl': 'https://www.vrt.be/vrtmax/a-z/radio-2-aan-zee/2024/radio-2-aan-zee-s2024a20/',
  'mediacontent_imageurl': '//images.vrt.be/orig/2024/07/20/bf8014c0-4698-11ef-92ff-02b7b76bf47f.jpg',
  'mediacontent_programimageurl': '//images.vrt.be/orig/2024/07/08/4c714b97-3d3c-11ef-92ff-02b7b76bf47f.png'}}